In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
import plotly.express as px

sys.path.insert(0, str(Path("").resolve().parents[1]))
from utils.db_manager import MimicDBManager

In [ ]:
width = 1200
height = 600

In [ ]:
db_manager = MimicDBManager(port=5432)
db_manager.retrieve_table_names()

In [ ]:
[db_manager.count_rows(table_name=table) for table in db_manager.retrieve_table_names()[:10]]

In [ ]:
table_name = db_manager.retrieve_table_names()[0]
db_manager.retrieve_id(table_name, id=18207287, id_column="subject_id")

In [ ]:
pd.DataFrame.from_dict(db_manager.retrieve_column_types(table_name), orient="index", columns=["data_type"]).reset_index(
    names="column"
)

In [ ]:
pd.DataFrame.from_dict(
    {table: db_manager.retrieve_column_types(table) for table in db_manager.retrieve_table_names()}, orient="index"
).fillna("-")

Let's look now for patients with Acute Renal Failure.

Note: ICD-9 Diagnostic codes can be found in: https://www2.gov.bc.ca/assets/gov/health/practitioner-pro/medical-services-plan/diag-codes_genitourinary.pdf

In [ ]:
data = db_manager.retrieve_all("mimiciv_hosp.diagnoses_icd")
data["icd_code"] = data["icd_code"].str.strip()
arf = data[data.icd_code == "5848"]
arf

In [ ]:
aki_patients = arf.merge(db_manager.retrieve_all("mimiciv_hosp.patients"), on="subject_id")
aki_patients

In [ ]:
fig = px.histogram(
    aki_patients,
    x="anchor_age",
    color="gender",
    barmode="overlay",
    marginal="box",
    text_auto=True,
    width=width,
    height=height,
)
fig.show()

Not too many patients to work with. How about patients with *any* type of acute renal failure

In [ ]:
data = db_manager.retrieve_all("mimiciv_hosp.diagnoses_icd")
data["icd_code"] = data["icd_code"].str.strip()
arf = data[data.icd_code.str.startswith("584")]
arf

In [ ]:
aki_patients = (
    arf.merge(db_manager.retrieve_all("mimiciv_hosp.patients"), on="subject_id")
    .merge(db_manager.retrieve_all("mimiciv_icu.icustays"), on="subject_id")
    .merge(db_manager.retrieve_all("mimiciv_hosp.admissions"), on="subject_id")
)
aki_patients["deceased"] = aki_patients["dod"].apply(lambda x: True if str(x).strip() != "None" else False)
aki_patients["icu_stay_days"] = (aki_patients["outtime"] - aki_patients["intime"]).dt.total_seconds() / 3600 / 24
aki_patients

In [ ]:
fig = px.histogram(
    aki_patients,
    x="anchor_age",
    facet_row="gender",
    barmode="group",
    marginal="box",
    color="deceased",
    nbins=30,
    text_auto=True,
    width=width,
    height=height,
)
fig.show()

In [ ]:
fig = px.histogram(
    aki_patients,
    x="anchor_age",
    # facet_row="race",
    barmode="group",
    color="race",
    nbins=20,
    text_auto=True,
    width=width,
    height=height,
)
fig.show()

91 is the hard limit for age on the DB, to guarantee non-identifiying patients, which would explain the higher amount of patients (both alive and deceased) in the last age group.

Let's take a look at the ICD codes as well

In [ ]:
fig = px.histogram(
    aki_patients,
    x="anchor_age",
    barmode="group",
    marginal="box",
    color="icd_code",
    nbins=10,
    text_auto=True,
    width=width,
    height=height,
)
fig.show()

In [ ]:
fig = px.histogram(
    aki_patients,
    x="icu_stay_days",
    barmode="group",
    marginal="box",
    color="icd_code",
    nbins=50,
    text_auto=True,
    width=width,
    height=height,
)
fig.show()